In [ ]:
# 1. Install required libraries

!pip install pycocotools requests albumentations ultralytics matplotlib pandas

!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
# 2. Import required libraries

import os
import zipfile
import requests
import shutil
import albumentations as A
import cv2
import glob
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import random


from tqdm import tqdm
from pycocotools.coco import COCO
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor, as_completed
from albumentations.pytorch import ToTensorV2

import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
True
1
NVIDIA L4


In [ ]:
# 3. Set up the main data directory

dataset_dir = '/content/custom_dataset'

# Ensure that the directory exists before downloading the dataset
os.makedirs(dataset_dir, exist_ok=True)

In [ ]:
# 4. Download and prepare custom dataset

print("Downloading custom dataset...")
custom_data_url = 'https://universe.roboflow.com/ds/Cs162RZWTo?key=lhgZJythPK'
custom_zip_path = f'{dataset_dir}/Traffic_Light.zip'
if not os.path.exists(custom_zip_path):
    response = requests.get(custom_data_url, stream=True)
    with open(custom_zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    print("Download complete.")

    # Extract custom dataset
    print("Extracting custom dataset...")
    with zipfile.ZipFile(custom_zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
    print("Extraction complete.")
else:
    print("Custom dataset already exists.")

# Map class IDs for custom dataset: 0 -> green pedestrian light, 1 -> red pedestrian light
# Label `2` (traffic light) will be ignored and corresponding images deleted
class_mapping = {0: 0, 1: 1}  # Only keep classes 0 and 1

# Function to filter out unwanted labels (e.g., traffic_light) and remove images without valid labels
def filter_and_adjust_annotations_and_images(images_dir, annotations_dir, class_mapping):
    """
    Filters out unwanted labels (e.g., traffic_light) and removes images without valid labels.
    """
    for label_filename in os.listdir(annotations_dir):
        if label_filename.endswith('.txt'):
            label_path = os.path.join(annotations_dir, label_filename)
            with open(label_path, 'r') as f:
                lines = f.readlines()

            # Filter and remap valid labels
            new_lines = []
            for line in lines:
                tokens = line.strip().split()
                current_class = int(tokens[0])
                if current_class in class_mapping:
                    tokens[0] = str(class_mapping[current_class])  # Remap class if necessary
                    new_lines.append(' '.join(tokens) + '\n')

            # If no valid labels remain, delete the label file and corresponding image
            if not new_lines:
                os.remove(label_path)
                # Delete the corresponding image file
                image_filename = os.path.splitext(label_filename)[0] + '.jpg'
                image_path = os.path.join(images_dir, image_filename)
                if os.path.exists(image_path):
                    os.remove(image_path)
                    print(f"Deleted image and label: {image_path}, {label_path}")
            else:
                # Save the filtered labels
                with open(label_path, 'w') as f:
                    f.writelines(new_lines)

# Apply label filtering and adjustment
# Apply filtering to train, valid, and test datasets
for split in ['train', 'valid', 'test']:
    filter_and_adjust_annotations_and_images(
        f'{dataset_dir}/{split}/images',
        f'{dataset_dir}/{split}/labels',
        class_mapping
    )

Download complete.
Extracting custom dataset...
Extraction complete.
Deleted image and label: /content/custom_dataset/train/images/Tlight-748-_png_jpg.rf.b4fd134591721dbf23f5eff0aa2b73b1.jpg, /content/custom_dataset/train/labels/Tlight-748-_png_jpg.rf.b4fd134591721dbf23f5eff0aa2b73b1.txt
Deleted image and label: /content/custom_dataset/train/images/Tlight-797-_png_jpg.rf.0dc0df06c38a4b43d7721a7faf1c1a2d.jpg, /content/custom_dataset/train/labels/Tlight-797-_png_jpg.rf.0dc0df06c38a4b43d7721a7faf1c1a2d.txt
Deleted image and label: /content/custom_dataset/train/images/tflight_001659_jpg.rf.4afb57a7c10c6f495773164e3dc677cf.jpg, /content/custom_dataset/train/labels/tflight_001659_jpg.rf.4afb57a7c10c6f495773164e3dc677cf.txt
Deleted image and label: /content/custom_dataset/train/images/Tlight-670-_png_jpg.rf.e5eafb96b9acfce531ee5c9f29ef02c5.jpg, /content/custom_dataset/train/labels/Tlight-670-_png_jpg.rf.e5eafb96b9acfce531ee5c9f29ef02c5.txt
Deleted image and label: /content/custom_dataset/trai

In [ ]:
# Define dataset directory
dataset_dir = '/content/custom_dataset'
os.makedirs(dataset_dir, exist_ok=True)

# List of files to download and extract
downloads = [
    {
        'url': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip',
        'dest': f'{dataset_dir}/annotations_trainval2017.zip',
        'extract_dir': f'{dataset_dir}/annotations'
    },
    {
        'url': 'http://images.cocodataset.org/zips/train2017.zip',
        'dest': f'{dataset_dir}/train2017.zip',
        'extract_dir': f'{dataset_dir}/train2017'
    },
    {
        'url': 'http://images.cocodataset.org/zips/val2017.zip',
        'dest': f'{dataset_dir}/val2017.zip',
        'extract_dir': f'{dataset_dir}/val2017'
    }
]

# Download function with file existence check
def download_file(url, destination):
    if os.path.exists(destination):  # Check if file already exists
        print(f"File already exists: {destination}")
        return
    print(f"Downloading {url}...")
    with requests.get(url, stream=True) as response:
        total_size = int(response.headers.get('content-length', 0))
        with open(destination, 'wb') as f, tqdm(
            desc=f"Downloading {os.path.basename(destination)}",
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
                bar.update(len(chunk))
    print(f"Downloaded {destination}.")

# Extract function with directory existence check
def extract_zip(source, destination):
    if os.path.exists(destination):  # Check if directory already exists
        print(f"Directory already exists: {destination}")
        return
    print(f"Extracting {source}...")
    with zipfile.ZipFile(source, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        with tqdm(total=len(file_list), desc=f"Extracting {os.path.basename(source)}") as bar:
            for file in file_list:
                zip_ref.extract(file, destination)
                bar.update(1)
    print(f"Extracted to {destination}.")

# Sequential download and extraction
print("Starting download and extraction...")
for item in downloads:
    try:
        # Download file
        download_file(item['url'], item['dest'])
        # Extract file
        extract_zip(item['dest'], item['extract_dir'])
    except Exception as e:
        print(f"Error processing {item['url']}: {e}")

print("All downloads and extractions complete.")

Starting download and extraction...


Downloaded /content/custom_dataset/annotations_trainval2017.zip.
Extracting /content/custom_dataset/annotations_trainval2017.zip...


Extracting annotations_trainval2017.zip: 100%|██████████| 6/6 [00:06<00:00,  1.11s/it]


Extracted to /content/custom_dataset/annotations.


Downloaded /content/custom_dataset/train2017.zip.
Extracting /content/custom_dataset/train2017.zip...


Extracting train2017.zip: 100%|██████████| 118288/118288 [01:17<00:00, 1521.63it/s]


Extracted to /content/custom_dataset/train2017.


Downloaded /content/custom_dataset/val2017.zip.
Extracting /content/custom_dataset/val2017.zip...


Extracting val2017.zip: 100%|██████████| 5001/5001 [00:02<00:00, 2300.81it/s]

Extracted to /content/custom_dataset/val2017.
All downloads and extractions complete.


In [ ]:
# Set dataset paths
dataset_dir = "/content/custom_dataset"
train_images_dir = os.path.join(dataset_dir, "train2017", "train2017")  # Nested train2017 folder
val_images_dir = os.path.join(dataset_dir, "val2017", "val2017")        # Nested val2017 folder

# Load COCO datasets
coco_train = COCO(os.path.join(dataset_dir, 'annotations', 'annotations', 'instances_train2017.json'))
coco_val = COCO(os.path.join(dataset_dir, 'annotations', 'annotations', 'instances_val2017.json'))

# Filter classes and create class ID mapping
coco_classes = ['bicycle', 'car', 'motorcycle', 'bus']  # Target classes
category_ids = coco_train.getCatIds(catNms=coco_classes)  # Get category IDs for selected classes
category_mapping = {cat_id: idx + 2 for idx, cat_id in enumerate(category_ids)}  # Map category IDs to YOLO IDs (starting from 2)

# Function to filter image IDs containing the target categories
def collect_filtered_ids(coco, category_ids):
    filtered_ids = set()  # Use a set to avoid duplicate IDs
    for cat_id in category_ids:
        img_ids = coco.getImgIds(catIds=[cat_id])  # Get image IDs for the category
        filtered_ids.update(img_ids)  # Add image IDs to the set
    return list(filtered_ids)

# Filter image IDs for train and validation datasets
filtered_train_ids = collect_filtered_ids(coco_train, category_ids)
filtered_val_ids = collect_filtered_ids(coco_val, category_ids)

# Convert COCO annotations to YOLO format
def convert_to_yolo_format(coco, img_id, category_ids, category_mapping):
    annotations = coco.loadAnns(coco.getAnnIds(imgIds=img_id, catIds=category_ids))  # Load annotations for the image
    img_info = coco.loadImgs(img_id)[0]  # Load image info
    width, height = img_info['width'], img_info['height']  # Image dimensions
    yolo_annotations = []

    for ann in annotations:
        cat_id = ann['category_id']  # COCO category ID
        yolo_class_id = category_mapping[cat_id]  # Map to YOLO class ID
        bbox = ann['bbox']  # Bounding box in COCO format [x_min, y_min, width, height]
        x_center = (bbox[0] + bbox[2] / 2) / width  # YOLO x_center (normalized)
        y_center = (bbox[1] + bbox[3] / 2) / height  # YOLO y_center (normalized)
        w = bbox[2] / width  # YOLO width (normalized)
        h = bbox[3] / height  # YOLO height (normalized)
        yolo_annotations.append(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")  # YOLO format

    return yolo_annotations

# Save COCO data in YOLO format
def save_coco_data(coco, img_ids, image_dir, label_dir, images_source_dir):
    os.makedirs(image_dir, exist_ok=True)  # Create image directory if it doesn't exist
    os.makedirs(label_dir, exist_ok=True)  # Create label directory if it doesn't exist

    for img_id in img_ids:
        img_info = coco.loadImgs(img_id)[0]  # Load image info
        file_name = img_info['file_name']  # Get image file name
        source_image_path = os.path.join(images_source_dir, file_name)  # Source image path
        target_image_path = os.path.join(image_dir, file_name)  # Target image path

        # Copy image file if it exists
        if os.path.exists(source_image_path):
            shutil.copy(source_image_path, target_image_path)
        else:
            print(f"[WARNING] Image {source_image_path} does not exist. Skipping...")  # Log missing images
            continue

        # Generate YOLO annotations and save to file
        yolo_annotations = convert_to_yolo_format(coco, img_id, category_ids, category_mapping)
        label_path = os.path.join(label_dir, f"{os.path.splitext(file_name)[0]}.txt")  # YOLO label file path
        with open(label_path, 'w') as f:
            f.writelines(yolo_annotations)  # Write annotations to file

# Save filtered train and validation datasets
print("Saving filtered COCO train data...")
save_coco_data(coco_train, filtered_train_ids, os.path.join(dataset_dir, 'coco', 'train', 'images'), os.path.join(dataset_dir, 'coco', 'train', 'labels'), train_images_dir)

print("Saving filtered COCO validation data...")
save_coco_data(coco_val, filtered_val_ids, os.path.join(dataset_dir, 'coco', 'valid', 'images'), os.path.join(dataset_dir, 'coco', 'valid', 'labels'), val_images_dir)

print("Data saved successfully.")


loading annotations into memory...
Done (t=18.66s)
creating index...
index created!
loading annotations into memory...
Done (t=0.61s)
creating index...
index created!
Saving filtered COCO train data...
Saving filtered COCO validation data...
Data saved successfully.


In [ ]:
# 6. Merge datasets

def merge_datasets(source_dir, target_dir):
    for split in ['images', 'labels']:
        src_split_dir = os.path.join(source_dir, split)
        tgt_split_dir = os.path.join(target_dir, split)
        os.makedirs(tgt_split_dir, exist_ok=True)
        for file in os.listdir(src_split_dir):
            shutil.copy(os.path.join(src_split_dir, file), tgt_split_dir)

print("Merging COCO train data with custom train data...")
merge_datasets(f'{dataset_dir}/coco/train', f'{dataset_dir}/train')
print("Merging COCO validation data with custom validation data...")
merge_datasets(f'{dataset_dir}/coco/valid', f'{dataset_dir}/valid')

Merging COCO train data with custom train data...
Merging COCO validation data with custom validation data...


In [ ]:
# 7. Create data.yaml

print("Creating data.yaml...")
data_yaml = {
    'train': f"{dataset_dir}/train/images",
    'val': f"{dataset_dir}/valid/images",
    'test': f"{dataset_dir}/test/images",
    'names': ['green pedestrian light', 'red pedestrian light'] + coco_classes,
    'nc': len(['green pedestrian light', 'red pedestrian light'] + coco_classes)
}
with open(f"{dataset_dir}/data.yaml", 'w') as f:
    yaml.dump(data_yaml, f)
print("Dataset preparation complete.")

Creating data.yaml...
Dataset preparation complete.


In [ ]:
# Debugging function to count files in a directory
def count_files_in_directory(directory, file_extension):
    return len([f for f in os.listdir(directory) if f.endswith(file_extension)])

# Debugging 1: Check filtered Roboflow dataset
def debug_roboflow_dataset(dataset_dir):
    print("Debugging Roboflow dataset:")
    for split in ['train', 'valid', 'test']:
        img_dir = os.path.join(dataset_dir, split, 'images')
        label_dir = os.path.join(dataset_dir, split, 'labels')
        print(f"  {split.capitalize()} - Images: {count_files_in_directory(img_dir, '.jpg')}, Labels: {count_files_in_directory(label_dir, '.txt')}")

debug_roboflow_dataset(dataset_dir)

# Debugging 2: Check COCO dataset filtering
def debug_coco_dataset(coco, category_ids, image_dir, label_dir):
    print("Debugging COCO dataset filtering:")
    filtered_img_count = len(coco.getImgIds(catIds=category_ids))
    img_count = count_files_in_directory(image_dir, '.jpg')
    label_count = count_files_in_directory(label_dir, '.txt')
    print(f"  Filtered Images (expected): {filtered_img_count}")
    print(f"  Saved Images: {img_count}, Saved Labels: {label_count}")

debug_coco_dataset(coco_train, category_ids, f'{dataset_dir}/coco/train/images', f'{dataset_dir}/coco/train/labels')

# Debugging 3: Check class ID mapping
print("Debugging class mapping:")
print(f"  Roboflow classes: {class_mapping}")
print(f"  COCO classes: {category_mapping}")

# Debugging 4: Verify final dataset structure
def debug_final_dataset(dataset_dir):
    print("Debugging final dataset structure:")
    for split in ['train', 'valid', 'test']:
        img_dir = os.path.join(dataset_dir, split, 'images')
        label_dir = os.path.join(dataset_dir, split, 'labels')
        print(f"  {split.capitalize()} - Images: {count_files_in_directory(img_dir, '.jpg')}, Labels: {count_files_in_directory(label_dir, '.txt')}")

debug_final_dataset(dataset_dir)

# Debugging 5: Inspect a random label file
def inspect_random_label_file(label_dir):
    label_files = glob.glob(os.path.join(label_dir, '*.txt'))
    if label_files:
        random_label_file = random.choice(label_files)
        print(f"Inspecting random label file: {random_label_file}")
        with open(random_label_file, 'r') as f:
            print(f.read())
    else:
        print(f"No label files found in {label_dir}.")

inspect_random_label_file(f'{dataset_dir}/train/labels')
inspect_random_label_file(f'{dataset_dir}/valid/labels')
inspect_random_label_file(f'{dataset_dir}/test/labels')

Debugging Roboflow dataset:
  Train - Images: 18800, Labels: 18800
  Valid - Images: 908, Labels: 908
  Test - Images: 41, Labels: 41
Debugging COCO dataset filtering:
  Filtered Images (expected): 53
  Saved Images: 17774, Saved Labels: 17774
Debugging class mapping:
  Roboflow classes: {0: 0, 1: 1}
  COCO classes: {2: 2, 3: 3, 4: 4, 6: 5}
Debugging final dataset structure:
  Train - Images: 18800, Labels: 18800
  Valid - Images: 908, Labels: 908
  Test - Images: 41, Labels: 41
Inspecting random label file: /content/custom_dataset/train/labels/000000456478.txt
4 0.216260 0.702988 0.237960 0.489880
4 0.089530 0.707763 0.051460 0.234745

Inspecting random label file: /content/custom_dataset/valid/labels/000000176037.txt
5 0.635258 0.527042 0.413172 0.461926
3 0.909555 0.541148 0.078328 0.040905
3 0.940695 0.548701 0.086672 0.045476
3 0.072281 0.527216 0.019938 0.049513
3 0.862773 0.539977 0.035891 0.041346
3 0.374523 0.498086 0.016266 0.017517

Inspecting random label file: /content/cus

In [ ]:
# 8. Delete Unnecessaries and Adjust Labels

# Delete unnecessary directories
dirs_to_remove = [
    f'{dataset_dir}/annotations',
    f'{dataset_dir}/coco',
    f'{dataset_dir}/train2017',
    f'{dataset_dir}/val2017'
]

for dir_path in dirs_to_remove:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"{dir_path} directories are deleted.")

# Delete unnecessary files
files_to_remove = [
    f'{dataset_dir}/README.dataset.txt',
    f'{dataset_dir}/README.roboflow.txt',
    f'{dataset_dir}/Traffic_Light.zip',
    f'{dataset_dir}/annotations_trainval2017.zip',
    f'{dataset_dir}/train2017.zip'
    f'{dataset_dir}/val2017.zip'
]

for file_path in files_to_remove:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} files are deleted.")

/content/custom_dataset/annotations directories are deleted.
/content/custom_dataset/coco directories are deleted.
/content/custom_dataset/train2017 directories are deleted.
/content/custom_dataset/val2017 directories are deleted.
/content/custom_dataset/README.dataset.txt files are deleted.
/content/custom_dataset/README.roboflow.txt files are deleted.
/content/custom_dataset/Traffic_Light.zip files are deleted.
/content/custom_dataset/annotations_trainval2017.zip files are deleted.


In [ ]:
# 10. Create data.yaml

print("Creating data.yaml...")
data_yaml = {
    'train': f"{dataset_dir}/train/images",
    'val': f"{dataset_dir}/valid/images",
    'test': f"{dataset_dir}/test/images",
    'names': ['green pedestrian light', 'red pedestrian light'] + coco_classes,
    'nc': len(['green pedestrian light', 'red pedestrian light'] + coco_classes)
}
data_yaml_path = f'{dataset_dir}/data.yaml'
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)
print("data.yaml created.")

Creating data.yaml...
data.yaml created.


In [ ]:
# 11. Train YOLOv8 model

# Load YOLOv8 model (Choose model version: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x)
model = YOLO('yolov8n.pt')

print("Starting YOLOv8 model training...")
model.train(
    data=data_yaml_path,    # Path to the data.yaml file
    epochs=50,              # Number of training epochs
    batch=16,               # Batch size
    imgsz=640,              # Input image size
    augment=True,           # Enable default augmentation
    degrees=10.0,           # Random rotation range
    translate=0.2,          # Random translation fraction
    scale=0.2,              # Random scale adjustment
    shear=2.0,              # Random shear range
    flipud=0.3,             # Upside-down flip probability
    fliplr=0.3,              # Left-right flip probability
    workers=8,
    amp=True,
    device=0
)

print("YOLOv8 model training complete.")

100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]


Starting YOLOv8 model training...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/custom_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

100%|██████████| 755k/755k [00:00<00:00, 20.4MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 104MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      2.05G      1.259      1.103       1.23         62        640: 100%|██████████| 1175/1175 [01:50<00:00, 10.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:03<00:00,  7.59it/s]


                   all        908       3020      0.784      0.691      0.747      0.502

50 epochs completed in 1.643 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.82 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.93it/s]


                   all        908       3020      0.802      0.663       0.74      0.496
green pedestrian light         74         74      0.946      0.946      0.939       0.62
  red pedestrian light         42         42      0.878      0.929      0.974      0.637
               bicycle        149        316      0.691      0.382      0.477      0.276
                   car        535       1932      0.735      0.504      0.606      0.391
            motorcycle        159        371      0.792      0.542       0.68      0.438
                   bus        189        285      0.771      0.674      0.763      0.612
Speed: 0.1ms preprocess, 3.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train
YOLOv8 model training complete.
